<a href="https://colab.research.google.com/github/wesnasimone/EA044-Algoritmos_Otimizacao/blob/main/Algoritmo_Branch_and_Bound_Wesna_225843.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##################################################

    Nome: Wesna Simone Bulla de Araujo
    RA: 225843
    Curso: EA044 - A
##################################################

##Algoritmo Branch and Bound (B&B) 🌳

In [ ]:
#Bibliotecas
import numpy as np
import pandas as pd
from scipy.optimize import linprog
import copy

### Função B&B

In [ ]:
#Ramificações do algoritmo Branch and Bound para achar soluções inteiras
#x_res: vetor de resultados da ramificação zero
#LS: valor ótimo obtido para a solução não inteira da ramificação zero
#LI: valor limite inferior para a solução ótima inteira
#vetor_c: vetor com as constantes da função objetivo
#vetor_r: vetor de listas com as constantes de todas as restrições
#vetor_b: vetor com os valores do lado direito da igualdade das restrições
#limites_var: vetor de listas dos limites inferios/superios das variaveis do problema

def branch_and_bound(x_res, LS, LI, vetor_c, vetor_r, vetor_b, limites_var):

  vetor_r_geral = [[],[]]
  vetor_b_geral = [[],[]]
  vetor_r1 = copy.deepcopy(vetor_r)
  vetor_r2 = copy.deepcopy(vetor_r)
  vetor_b1 = copy.deepcopy(vetor_b)
  vetor_b2 = copy.deepcopy(vetor_b)

  flag_branch = 0
  solucao = True
  count_bb = 0
  while solucao:

    nova_restricao = []
    vetor_r_aux1 = []
    vetor_r_aux2 = []
    vetor_b_aux1 = []
    vetor_b_aux2 = []
    lista_nova_restricao = []
    lista_nova_restricao1 = []

    #Define novas limitações para as variaveis
    for c in range(qtd_c):
      decimal = int(str(round(x_res[c],2)).split('.')[1])
      nova_restricao.append(abs(50 - decimal))                      # escolhe a variável cuja parte decimal é mais próxima de 0.5
      indice = nova_restricao.index(min(nova_restricao))
    x_nova_restricao = round(x_res[indice],2)

    #Define intervalos para o novo valor de x
    x_res1 = int(x_nova_restricao)
    x_res2 = x_res1 + 1

    #Prepara novas restrições com base na nova variável em análise (2 ramificações)
    for c in range(qtd_c):
        if indice == c:
          lista_nova_restricao.append(1)          # equação menor que
          lista_nova_restricao1.append(-1)        # equação maior que
        else:
          lista_nova_restricao.append(0)
          lista_nova_restricao1.append(0)

    #Inicia as ramificações para as duas novas condições
    #Branch 0
    if (flag_branch == 0):
      #Adiciona as novas restrições ao problema antigo inicial
      vetor_r1.append(lista_nova_restricao)
      vetor_b1.append(x_res1)
      vetor_r2.append(lista_nova_restricao1)
      vetor_b2.append(x_res2*(-1))

      #Adiciona a nova branch (as duas ramificações geradas) em um vetor global
      vetor_r_geral[0].append(vetor_r1)
      vetor_b_geral[0].append(vetor_b1)
      vetor_r_geral[1].append(vetor_r2)
      vetor_b_geral[1].append(vetor_b2)

      #Usa o algoritmo simplex para achar uma outra solução com base nos novos valores das ramificações
      res1 = linprog(vetor_c, A_ub=vetor_r1, b_ub=vetor_b1, bounds=limites_var, method='simplex')
      res2 = linprog(vetor_c, A_ub=vetor_r2, b_ub=vetor_b2, bounds=limites_var, method='simplex')
    #Branch 1,2...
    else:
      #Se a ramificação anterior surgiu da primeira possibilidade de ramificação
      if flag_branch == 1:
          vetor_r_aux1 = copy.deepcopy(vetor_r_geral[0][(len(vetor_r_geral[0]))-1])
          vetor_b_aux1 = copy.deepcopy(vetor_b_geral[0][(len(vetor_r_geral[0]))-1])
          vetor_r_aux2 = copy.deepcopy(vetor_r_geral[0][(len(vetor_r_geral[0]))-1])
          vetor_b_aux2 = copy.deepcopy(vetor_b_geral[0][(len(vetor_r_geral[0]))-1])

      #Se a ramificação anterior surgiu da segunda possibilidade de ramificação
      else:
          vetor_r_aux1 = copy.deepcopy(vetor_r_geral[1][(len(vetor_r_geral[0]))-1])
          vetor_b_aux1 = copy.deepcopy(vetor_b_geral[1][(len(vetor_r_geral[0]))-1])
          vetor_r_aux2 = copy.deepcopy(vetor_r_geral[1][(len(vetor_r_geral[0]))-1])
          vetor_b_aux2 = copy.deepcopy(vetor_b_geral[1][(len(vetor_r_geral[0]))-1])

      #Dependendo da ramificação adiciona às restrições anteriores a nova restrição
      vetor_r_aux1.append(lista_nova_restricao)
      vetor_b_aux1.append(x_res1)
      vetor_r_aux2.append(lista_nova_restricao1)
      vetor_b_aux2.append(x_res2*(-1))

      #Armazena as novas restrições no vetor global para ser utilizado nas próximas ramificações
      vetor_r_geral[0].append(vetor_r_aux1)
      vetor_b_geral[0].append(vetor_b_aux1)
      vetor_r_geral[1].append(vetor_r_aux2)
      vetor_b_geral[1].append(vetor_b_aux2)

      #Usa o algoritmo simplex para achar uma outra solução com base nos novos valores das ramificações
      res1 = linprog(vetor_c, A_ub=vetor_r_aux1, b_ub=vetor_b_aux1, bounds=limites_var, method='simplex')
      res2 = linprog(vetor_c, A_ub=vetor_r_aux2, b_ub=vetor_b_aux2, bounds=limites_var, method='simplex')

    ###############################################################################################

    # Análise das soluções encontradas #

    #Contador da quantidade de vezes que foi necessário uma ramificação
    count_bb = count_bb + 1

    count1 = 0
    count2 = 0
    #define se existe alguma solução inteira
    for c in range(qtd_c):
      if (res1.x[c] == int(res1.x[c])):
        count1 = count1 + 1
      if (res2.x[c] == int(res2.x[c])):
        count2 = count2 + 1

    # Se a primeira possível ramificação ou a segunda possível ramificação deram valores inteiros
    if (count1 == qtd_c) or (count2 == qtd_c):
      # Se a primeira ramificação deu solução inteira
      if count1 == qtd_c:
        # verifica se o valor da função objetivo é menor que o limite superior e maior que o limite inferior
        if (abs(res1.fun) < LS) and (abs(res1.fun) > LI):
          # verifica se o valor da função objetivo é maior que o valor da função objetivo da segunda ramificação
          # ou se o valor da função objetivo da segunda ramificação é maior que o limite superior
          if (abs(res1.fun) >= abs(res2.fun)) or (abs(res2.fun) >= LS):
            # verifica se a ramificação dois é inteira, se seu valor é maior que o valor da ramificação um e se
            # a ramificação dois respeita os limites inferior e superior
            if ((count2 == qtd_c) and (abs(res2.fun) >= abs(res1.fun)) and (abs(res2.fun) < LS) and (abs(res2.fun) > LI)):
              #se sim, a solução proveniente da ramificação dois é a melhor solução inteira
              x_vetor = res2.x
              v_fun = res2.fun
              ciclo = 2*(count_bb)+1
              solucao = False
            else:
              #se não, a solução um de fato é a melhor solução inteira
              x_vetor = res1.x
              v_fun = res1.fun
              ciclo = 2*(count_bb)+1
              solucao = False
          else:
            # se a ramificação 1 não coincidir com todos os critérios
            # analisa-se a ramificação dois, pode ela ainda pode ser inteira
            if ((count2 == qtd_c) and (abs(res2.fun) < LS) and (abs(res2.fun) > LI)):
              #se atender a todos os critérios a solução da ramificação dois é a resposta inteira buscada
              x_vetor = res2.x
              v_fun = res2.fun
              ciclo = 2*(count_bb)+1
              solucao = False
            else:
              #se não, verifica os valores dos limites para atualizar ou não o limite inferior
              if (abs(res1.fun) < LS) and (abs(res1.fun) > LI):
                LI = abs(res1.fun)
              #faz da próxima variável o valor obtido pela ramificação dois
              x_res = res2.x[:]
              flag_branch = 2
        else:
          #solução da ramificação 1 não atende aos critérios do problema
          #checa se a ramificação dois pode ser inteira e se atende aos critérios do problema
          if ((count2 == qtd_c) and (abs(res2.fun) < LS) and (abs(res2.fun) > LI)):
              #se sim, a solução proveniente da ramificação dois é a melhor solução inteira
              x_vetor = res2.x
              v_fun = res2.fun
              ciclo = 2*(count_bb)+1
              solucao = False
          else:
            #se não, verifica os valores dos limites para atualizar ou não o limite inferior
            if (abs(res1.fun) < LS) and (abs(res1.fun) > LI):
              LI = abs(res1.fun)
            #faz da próxima variável o valor obtido pela ramificação dois
            x_res = res2.x[:]
            flag_branch = 2

      else:
        #Apenas a ramificação dois apresentou solução inteira
        # verifica se o valor da função objetivo é menor que o limite superior e maior que o limite inferior
        if (abs(res2.fun) < LS) and (abs(res2.fun) > LI):
          # verifica se o valor da função objetivo é maior que o valor da função objetivo da segunda ramificação
          # ou se o valor da função objetivo da segunda ramificação é maior que o limite superior
          if (abs(res2.fun) >= abs(res1.fun)) or (abs(res1.fun) >= LS):
            #se sim, a solução proveniente da ramificação dois é a melhor solução inteira
            x_vetor = res2.x
            v_fun = res2.fun
            ciclo = 2*(count_bb)+1
            solucao = False
          else:
            #se não, verifica os valores dos limites para atualizar ou não o limite inferior
            if (abs(res2.fun) < LS) and (abs(res2.fun) > LI):
              LI = abs(res2.fun)
            #faz da próxima variável o valor obtido pela ramificação um
            x_res = res1.x[:]
            flag_branch = 1
        else:
          #solução da ramificação 2 não atende aos critérios do problema
          #verifica os valores dos limites para atualizar ou não o limite inferior
          if (abs(res2.fun) < LS) and (abs(res2.fun) > LI):
            LI = abs(res2.fun)
          #faz da próxima variável o valor obtido pela ramificação um
          x_res = res1.x[:]
          flag_branch = 1
    else:
      #Nenhuma solução inteira foi encontrada
      #Escolhe aquela ramificação que possui solução mais próxima de LS
      dif_LS = []
      dif_LS.append(abs(LS - abs(res1.fun)))
      dif_LS.append(abs(LS - abs(res2.fun)))
      indice = dif_LS.index(min(dif_LS))
      if indice == 0:
        #faz da próxima variável o valor obtido pela ramificação um
        x_res = res1.x[:]
        flag_branch = 1
      else:
        #faz da próxima variável o valor obtido pela ramificação dois
        x_res = res2.x[:]
        flag_branch = 2
  return x_vetor, v_fun, ciclo  #retorna os valores das variaveis, o valor da solução final e a quantidade de ramificações

##Entradas

Observações 👀

 - Caso necessário colocar as inequações (coeficiente das inequações) no formato de inequação menor ou igual. Ex: x + y > 2 deve ser transformado para -x - y < -2;

In [ ]:
#Coeficientes da função objetivo
qtd_c = int(input('Quantos coeficientes tem a sua função objetivo: '))
tipo_problema = input('Seu problema é de Minimização? Digite S ou N: ')

vetor_c = []
for c in range(qtd_c):
  valor_c = float(input(f'Digite o coeficiente x{c+1}: '))
  if tipo_problema.upper() == 'N':
    valor_c = valor_c*(-1)
  vetor_c.append(valor_c)

Quantos coeficientes tem a sua função objetivo: 2
Seu problema é de Minimização? Digite S ou N: n
Digite o coeficiente x1: 2
Digite o coeficiente x2: 1


In [ ]:
#Coeficientes das restrições (lado esquerdo da desigualdade ou igualdade)
qtd_r = int(input('Quantos restrições tem seu problema: '))

vetor_r = [[] for c in range(qtd_r)]
for c in range(qtd_r):
  print(f'------ Restrição {c+1} ------')
  for d in range(qtd_c):
    valor_r = float(input(f'Digite o coeficiente x{d+1}: '))
    vetor_r[c].append(valor_r)

Quantos restrições tem seu problema: 3
------ Restrição 1 ------
Digite o coeficiente x1: 1
Digite o coeficiente x2: 1
------ Restrição 2 ------
Digite o coeficiente x1: -1
Digite o coeficiente x2: 1
------ Restrição 3 ------
Digite o coeficiente x1: 6
Digite o coeficiente x2: 2


In [ ]:
#Coeficientes das restrições (lado direito da desigualdade ou igualdade)
vetor_b = []
for c in range(qtd_r):
  print(f'------ Restrição {c+1} ------')
  valor_b = float(input(f'Digite o coeficiente b{c+1}: '))
  vetor_b.append(valor_b)

------ Restrição 1 ------
Digite o coeficiente b1: 5
------ Restrição 2 ------
Digite o coeficiente b2: 0
------ Restrição 3 ------
Digite o coeficiente b3: 21


In [ ]:
#Limite inferior das variaveis do problema
limites_var = []
for c in range(qtd_c):
   valor_limite_inf = float(input(f'Digite o limite inferior x{c+1}: '))
   limites_var.append((valor_limite_inf, None))

Digite o limite inferior x1: 0
Digite o limite inferior x2: 0


In [ ]:
# Primeira solução do simplex
res = linprog(vetor_c, A_ub=vetor_r, b_ub=vetor_b, bounds=limites_var, method='simplex')
x_res = res.x[:]

<ipython-input-19-6e3b17c612fe>:2: DeprecationWarning: `method='simplex'` is deprecated and will be removed in SciPy 1.11.0. Please use one of the HiGHS solvers (e.g. `method='highs'`) in new code.
  res = linprog(vetor_c, A_ub=vetor_r, b_ub=vetor_b, bounds=limites_var, method='simplex')


##Em busca da solução inteira

In [ ]:
#Passo 1: verificar se a primeira solução encontrada já é inteira
count = 0
for c in range(qtd_c):
  if (x_res[c] == int(x_res[c])):
    count = count + 1
if count == qtd_c:
#Se a primeira solução já for inteira o problema está resolvido
  print(f'Solução ótima: {res.fun}')
  print(f'Vetor x: {x_res}')
  print(f'Ramificações B&B: {0}')
else:
#Se não --> Passo 2: algoritmo B&B

  #Define limite superior
  if tipo_problema.upper() == 'N':
    LS = res.fun*(-1)
  else:
    LS = res.fun
  LI = 0

  print('########################')
  x_vetor, v_fun, ciclo = branch_and_bound(x_res, LS, LI, vetor_c, vetor_r, vetor_b, limites_var) # calcula a solução inteira
  print(f'Solução ótima: {v_fun}')
  print(f'Vetor x: {x_vetor}')
  print(f'Ramificações B&B: {ciclo}')
  print('########################')


########################
Solução ótima: -7.0
Vetor x: [3. 1.]
Ramificações B&B: 7
########################


<ipython-input-2-775bd5fa38e0>:68: DeprecationWarning: `method='simplex'` is deprecated and will be removed in SciPy 1.11.0. Please use one of the HiGHS solvers (e.g. `method='highs'`) in new code.
  res1 = linprog(vetor_c, A_ub=vetor_r1, b_ub=vetor_b1, bounds=limites_var, method='simplex')
<ipython-input-2-775bd5fa38e0>:69: DeprecationWarning: `method='simplex'` is deprecated and will be removed in SciPy 1.11.0. Please use one of the HiGHS solvers (e.g. `method='highs'`) in new code.
  res2 = linprog(vetor_c, A_ub=vetor_r2, b_ub=vetor_b2, bounds=limites_var, method='simplex')
<ipython-input-2-775bd5fa38e0>:99: DeprecationWarning: `method='simplex'` is deprecated and will be removed in SciPy 1.11.0. Please use one of the HiGHS solvers (e.g. `method='highs'`) in new code.
  res1 = linprog(vetor_c, A_ub=vetor_r_aux1, b_ub=vetor_b_aux1, bounds=limites_var, method='simplex')
<ipython-input-2-775bd5fa38e0>:100: DeprecationWarning: `method='simplex'` is deprecated and will be removed in SciP